Group 13 | Aditya Sharma, Adarsh Balan, Thaneshwar Prasad Sahu, Muhammad Ashraf Hussain, Prathyusha Thatipelli
Part B | Question 1 and 2

In [3]:
# Connecting to Google Collab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Importing libraries
import os
import shutil
from zipfile import ZipFile

In [5]:
# Defining Function to extract zip file
def extract_zip(file_path, extract_to):
    with ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted to {extract_to}")

In [6]:
# Defining Function to split dataset into train and test
def split_dataset(dataset_path, train_dir, test_dir):
    categories = os.listdir(dataset_path)
    categories.sort()

    for category in categories:
        category_path = os.path.join(dataset_path, category)
        images = sorted(os.listdir(category_path))

        # Creating directories for train and test sets
        train_category_dir = os.path.join(train_dir, category)
        test_category_dir = os.path.join(test_dir, category)
        os.makedirs(train_category_dir, exist_ok=True)
        os.makedirs(test_category_dir, exist_ok=True)

        # Splitting and copying files
        for i, img in enumerate(images):
            src_path = os.path.join(category_path, img)
            if i < 40:  # First 40 images for training
                shutil.copy(src_path, train_category_dir)
            else:  # Remaining images for testing
                shutil.copy(src_path, test_category_dir)

    print(f"Training and testing datasets created at {train_dir} and {test_dir}")

In [7]:
# Files Paths
zip_file_path = '/content/drive/MyDrive/AAI Group Assignment/dataset.zip'
extract_to = '/content/drive/MyDrive/AAI Group Assignment/dataset'
train_dir = '/content/drive/MyDrive/AAI Group Assignment/train'
test_dir = '/content/drive/MyDrive/AAI Group Assignment/test'

# Extracting images dataset
extract_zip(zip_file_path, extract_to)

# Check the contents of the extracted directory
print("Contents of extracted directory:", os.listdir(extract_to))

# Splitting dataset
split_dataset(os.path.join(extract_to, 'dataset'), train_dir, test_dir)

# Check the contents of the train and test directories
print("Contents of train directory:", os.listdir(train_dir))
print("Contents of test directory:", os.listdir(test_dir))

Extracted to /content/drive/MyDrive/AAI Group Assignment/dataset
Contents of extracted directory: ['dataset']
Training and testing datasets created at /content/drive/MyDrive/AAI Group Assignment/train and /content/drive/MyDrive/AAI Group Assignment/test
Contents of train directory: ['accordion', 'bass', 'camera', 'crocodile', 'crocodile_head', 'cup', 'dollar_bill', 'emu', 'gramophone', 'hedgehog', 'nautilus', 'pizza', 'pyramid', 'sea_horse', 'windsor_chair']
Contents of test directory: ['accordion', 'bass', 'camera', 'crocodile', 'crocodile_head', 'cup', 'dollar_bill', 'emu', 'gramophone', 'hedgehog', 'nautilus', 'pizza', 'pyramid', 'sea_horse', 'windsor_chair']


In [8]:
# Importing Libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, classification_report

In [9]:
# Checking for GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
# Loading the dataset
train_dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/AAI Group Assignment/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/AAI Group Assignment/test', transform=transform)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
# Defining models to train
models_to_train = {
    'resnet18': models.resnet18(pretrained=True),
    'densenet121': models.densenet121(pretrained=True),
    'vgg19': models.vgg19(pretrained=True)
}

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 147MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

In [14]:
# Modifying the last layer for 15 classes
for model_name, model in models_to_train.items():
    if 'resnet' in model_name:
        num_ftrs = model.fc.in_features
        model.fc = torch.nn.Linear(num_ftrs, 15)
    elif 'vgg' in model_name:
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = torch.nn.Linear(num_ftrs, 15)
    elif 'densenet' in model_name:
        num_ftrs = model.classifier.in_features
        model.classifier = torch.nn.Linear(num_ftrs, 15)
    model.to(device)

In [15]:
# Defining Training and evaluation function
def train_and_evaluate(model, train_loader, test_loader, device):
    # Define loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

    # Evaluation
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision = precision_score(all_labels, all_preds, average=None)
    recall = recall_score(all_labels, all_preds, average=None)
    return precision, recall

In [20]:
# Training and evaluating each model
num_epochs = 25
results = {}

for model_name, model in models_to_train.items():
    print(f"Training and evaluating {model_name}...")
    precision, recall = train_and_evaluate(model, train_loader, test_loader, device)
    results[model_name] = {
        'precision': precision,
        'recall': recall
    }

Training and evaluating resnet18...
Epoch 1/25, Loss: 0.14063434302806854
Epoch 2/25, Loss: 0.057409320026636124
Epoch 3/25, Loss: 0.06868451833724976
Epoch 4/25, Loss: 0.037219174206256866
Epoch 5/25, Loss: 0.028178133070468903
Epoch 6/25, Loss: 0.06909924000501633
Epoch 7/25, Loss: 0.026505939662456512
Epoch 8/25, Loss: 0.02454829216003418
Epoch 9/25, Loss: 0.031990908086299896
Epoch 10/25, Loss: 0.022165261209011078
Epoch 11/25, Loss: 0.02897891402244568
Epoch 12/25, Loss: 0.01501509826630354
Epoch 13/25, Loss: 0.02373841218650341
Epoch 14/25, Loss: 0.018202783539891243
Epoch 15/25, Loss: 0.020262477919459343
Epoch 16/25, Loss: 0.040584344416856766
Epoch 17/25, Loss: 0.015272463671863079
Epoch 18/25, Loss: 0.017581652849912643
Epoch 19/25, Loss: 0.011857635341584682
Epoch 20/25, Loss: 0.011391010135412216
Epoch 21/25, Loss: 0.011890620924532413
Epoch 22/25, Loss: 0.01605450175702572
Epoch 23/25, Loss: 0.008651270531117916
Epoch 24/25, Loss: 0.022066481411457062
Epoch 25/25, Loss: 0.

In [21]:
# Get class names
class_names = train_dataset.classes

In [22]:
# Print results with class names
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print("Per-class Precision and Recall:")
    for i, class_name in enumerate(class_names):
        print(f"{class_name}: Precision - {metrics['precision'][i]:.4f}, Recall - {metrics['recall'][i]:.4f}")
    print()

Model: resnet18
Per-class Precision and Recall:
accordion: Precision - 1.0000, Recall - 1.0000
bass: Precision - 1.0000, Recall - 1.0000
camera: Precision - 1.0000, Recall - 1.0000
crocodile: Precision - 0.8000, Recall - 0.8000
crocodile_head: Precision - 0.8182, Recall - 0.8182
cup: Precision - 1.0000, Recall - 1.0000
dollar_bill: Precision - 1.0000, Recall - 1.0000
emu: Precision - 1.0000, Recall - 1.0000
gramophone: Precision - 1.0000, Recall - 1.0000
hedgehog: Precision - 0.9333, Recall - 1.0000
nautilus: Precision - 1.0000, Recall - 1.0000
pizza: Precision - 1.0000, Recall - 1.0000
pyramid: Precision - 1.0000, Recall - 1.0000
sea_horse: Precision - 1.0000, Recall - 0.9412
windsor_chair: Precision - 1.0000, Recall - 1.0000

Model: densenet121
Per-class Precision and Recall:
accordion: Precision - 1.0000, Recall - 1.0000
bass: Precision - 1.0000, Recall - 1.0000
camera: Precision - 1.0000, Recall - 1.0000
crocodile: Precision - 0.9000, Recall - 0.9000
crocodile_head: Precision - 0.9